Notebook notes: Not really an important notebook. Just one used to debug the functions used in training.py file

In [1]:
%load_ext autoreload
%autoreload 2
import torch
from PIL import Image
from torchvision.transforms import ToTensor
from torch.nn import BCEWithLogitsLoss
import numpy as np
import random
from tqdm import tqdm
import argparse
from datetime import date
from sklearn.metrics import confusion_matrix
from torch.utils.tensorboard import SummaryWriter
import sys
sys.path.insert(0, '../utils')
from ground_truth_dataset import groundTruthDataset
from data_functions import splitDataset, returnLoaders
from metrics import returnInterUnion, returnPreReF
sys.path.insert(0, '../models')
from unet_model import UNet
import matplotlib
import matplotlib.pyplot as plt
sys.path.insert(0, '../train')
#from training import train_one_epoch, valid_one_epoch 
matplotlib.rcParams['figure.figsize'] = (10, 6)
matplotlib.rcParams.update({'font.size': 18})

In [2]:
from training import train_one_epoch, valid_one_epoch 

In [3]:
NUM_CLASSES = 3 # For ground truth data, there's 3 classes of Background, Lagoon, CAFO
batch_size = 4

In [4]:
dataset = groundTruthDataset("../../../segmentation_ground_truth")
datasets = splitDataset(dataset)
trainloader, validloader, testloader = returnLoaders(datasets, 4, True)

In [5]:
model = UNet(3, 3) # 3 Channels, 3 Classes (background, lagoon, CAFO)

In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [7]:
train_num_batches = len(trainloader)
valid_num_batches = len(validloader)
train_num_examples = len(trainloader.dataset)
valid_num_examples = len(validloader.dataset)

In [8]:
#Set model to either cpu or gpu
model.to(device)            

#Define loss function
#Weight due to class imbalance
pos_weight = torch.tensor([50, 1000, 1])
pos_weight = torch.reshape(pos_weight,(1,3,1,1)).to(device)
criterion = BCEWithLogitsLoss(pos_weight=pos_weight)

In [9]:
optimizer = torch.optim.Adam(model.parameters(), lr = 1e-3, 
                                     weight_decay = 1e-6)

In [10]:
today = date.today()
date_prefix = today.strftime("%m_%d")
log_dir_suffix = f"{date_prefix}_groundtruth_lr_{3e-4}_epochs_{10}_batch_size_{4}"
log_dir = "../logs/groundtruth/" + log_dir_suffix
writer = SummaryWriter(log_dir=log_dir)

In [11]:
torch.manual_seed(0)
np.random.seed(0)
model.zero_grad()
for epoch in range(7):
    ### TRAINING ###
    print("Beginning Training in Epoch " + str(epoch))
    with tqdm(total = train_num_batches) as epoch_pbar:
        model.train()
        train_loss, train_correct, \
            train_IoU = train_one_epoch(epoch, train_num_batches, model, 
                                        device, trainloader, epoch_pbar, 
                                        optimizer, writer, criterion)

    ### VALIDATION ###
    print("Beginning Validation in Epoch " + str(epoch))
    valid_loss = []
    valid_correct = 0

    conf_matrix = np.zeros((3, 3))

    with tqdm(total = valid_num_batches) as epoch_pbar:
        model.eval()                           
        valid_loss, valid_correct, \
            conf_matrix, valid_IoU = valid_one_epoch(epoch, valid_num_batches, model, 
                                                     device, validloader, epoch_pbar, 
                                                     optimizer, writer, criterion,
                                                     conf_matrix)

  0%|          | 0/20 [00:00<?, ?it/s]

Beginning Training in Epoch 0


  0%|          | 0/20 [00:00<?, ?it/s]


RuntimeError: CUDA out of memory. Tried to allocate 256.00 MiB (GPU 0; 11.17 GiB total capacity; 4.70 GiB already allocated; 57.44 MiB free; 4.74 GiB reserved in total by PyTorch)

In [ ]:
precision, recall, f_score = returnPreReF(conf_matrix, 2)                
(2 * precision * recall)/(precision + recall)
conf_matrix

In [ ]:
# Testing it out on validation
val_example = next(iter(validloader))
inputs = val_example[0].to(device)
labels = val_example[1].to(device)
with torch.no_grad():
    outputs = model(inputs)
_, predictions = torch.max(outputs, 1)
predictions_one_hot = torch.nn.functional.one_hot(predictions).permute(0, 3, 1, 2)
y_pred = predictions.flatten().cpu().numpy()
_, blah = torch.max(labels, 1)
y_true = blah.flatten().cpu().numpy()

In [ ]:
plt.imshow(inputs[0].permute(1, 2, 0).cpu().numpy())

In [ ]:
def plotLabelPredict(inputs, label, predict):
    f = plt.figure()
    a = f.add_subplot(1, 2, 1)
    plt.imshow(inputs.permute(1, 2, 0).cpu().numpy())
    kwarg_dict = {'CAFO Shed': {'cmap':'prism', 'alpha': 0.5},
                  'Lagoon': {'cmap':'cool', 'alpha': 0.5},
                  'BACKGROUND': {'cmap':'hot', 'alpha': 0.0}}
    plt.imshow(label[0].cpu().numpy(), **kwarg_dict['BACKGROUND'])
    plt.imshow(label[1].cpu().numpy(), **kwarg_dict['Lagoon'])
    plt.imshow(label[2].cpu().numpy(), **kwarg_dict['CAFO Shed'])    
    plt.axis('off')
    a = f.add_subplot(1, 2, 2)
    plt.imshow(inputs.permute(1, 2, 0).cpu().numpy())
    plt.imshow(predict[0].cpu().numpy(), **kwarg_dict['BACKGROUND'])
    plt.imshow(predict[1].cpu().numpy(), **kwarg_dict['Lagoon'])
    plt.imshow(predict[2].cpu().numpy(), **kwarg_dict['CAFO Shed'])    
    plt.axis('off')
    plt.show()

In [ ]:
for i in range(4):
    plotLabelPredict(inputs[i], labels[i], predictions_one_hot[i])

In [ ]:
import cv2
planet_images_dir = '../../../notebooks/home/planet_images_il-old/'
pic_list = ['planet_loc_103_date_2020-04-29.tif', 'planet_loc_107_date_2020-04-29.tif', 'planet_loc_110_date_2020-04-29.tif',
           'planet_loc_112_date_2020-04-29.tif']
image_list = [planet_images_dir + i for i in pic_list]

In [ ]:
def planet_images(picture, model, device):
    img = cv2.imread(picture)
    img = cv2.cvtColor(img, cv2.COLOR_BGRA2RGB)
    example = Image.fromarray(img)
    example = ToTensor()(example)
    
    example_output = model((example.unsqueeze(0)).to(device))
    _, predictions = torch.max(example_output, 1)
    predictions_one_hot = torch.nn.functional.one_hot(predictions).permute(0, 3, 1, 2)
    
    f = plt.figure()
    a = f.add_subplot(1, 2, 1)
    plt.imshow(example.permute(1, 2, 0).cpu().numpy())
    kwarg_dict = {'CAFO Shed': {'cmap':'prism', 'alpha': 0.5},
                  'Lagoon': {'cmap':'cool', 'alpha': 0.5},
                  'BACKGROUND': {'cmap':'hot', 'alpha': 0.0}}
    plt.axis('off')
    a = f.add_subplot(1, 2, 2)
    plt.imshow(example.permute(1, 2, 0).cpu().numpy())
    plt.imshow(predictions_one_hot[0][0].cpu().numpy(), **kwarg_dict['BACKGROUND'])
    plt.imshow(predictions_one_hot[0][1].cpu().numpy(), **kwarg_dict['Lagoon'])
    plt.imshow(predictions_one_hot[0][2].cpu().numpy(), **kwarg_dict['CAFO Shed'])    
    plt.axis('off')
    plt.show()

In [ ]:
for i in image_list:
    planet_images(i, model, device)

In [ ]:
#103, 107, 110, 112

example1_dir = planet_images_dir + 'planet_loc_110_date_2020-04-29.tif'
img1 = cv2.imread(example1_dir)
img1 = cv2.cvtColor(img1, cv2.COLOR_BGRA2RGB)
example1 = Image.fromarray(img1)
example1 = ToTensor()(example1)
plt.imshow(example1.permute(1, 2, 0).cpu().numpy())

In [ ]:
example1_output = model((example1.unsqueeze(0)).to(device))
_, predictions1 = torch.max(example1_output, 1)
predictions_one_hot1 = torch.nn.functional.one_hot(predictions1).permute(0, 3, 1, 2)

In [ ]:
torch.sum(predictions_one_hot1[0], [1,2])

In [ ]:
f = plt.figure()
a = f.add_subplot(1, 2, 1)
plt.imshow(example1.permute(1, 2, 0).cpu().numpy())
kwarg_dict = {'CAFO Shed': {'cmap':'prism', 'alpha': 0.5},
              'Lagoon': {'cmap':'cool', 'alpha': 0.5},
              'BACKGROUND': {'cmap':'hot', 'alpha': 0.0}}
plt.axis('off')
a = f.add_subplot(1, 2, 2)
plt.imshow(example1.permute(1, 2, 0).cpu().numpy())
plt.imshow(predictions_one_hot1[0][0].cpu().numpy(), **kwarg_dict['BACKGROUND'])
plt.imshow(predictions_one_hot1[0][1].cpu().numpy(), **kwarg_dict['Lagoon'])
plt.imshow(predictions_one_hot1[0][2].cpu().numpy(), **kwarg_dict['CAFO Shed'])    
plt.axis('off')
plt.show()

In [ ]:
example_dir1 = planet_images_dir + 'planet_loc_103_date_2020-04-29.tif'
img1 = cv2.imread(example_dir1)
img1 = cv2.cvtColor(img1, cv2.COLOR_BGRA2RGB)
example1 = Image.fromarray(img1)
example1 = ToTensor()(example1)

example_dir2 = planet_images_dir + 'planet_loc_107_date_2020-04-29.tif'
img2 = cv2.imread(example_dir2)
img2 = cv2.cvtColor(img2, cv2.COLOR_BGRA2RGB)
example2 = Image.fromarray(img2)
example2 = ToTensor()(example2)

example_dir3 = planet_images_dir + 'planet_loc_110_date_2020-04-29.tif'
img3 = cv2.imread(example_dir3)
img3 = cv2.cvtColor(img3, cv2.COLOR_BGRA2RGB)
example3 = Image.fromarray(img3)
example3 = ToTensor()(example3)

example_dir4 = planet_images_dir + 'planet_loc_112_date_2020-04-29.tif'
img4 = cv2.imread(example_dir4)
img4 = cv2.cvtColor(img4, cv2.COLOR_BGRA2RGB)
example4 = Image.fromarray(img4)
example4 = ToTensor()(example4)

In [ ]:
import cv2

In [ ]:
example1.shape

In [ ]:
intersection = np.logical_and(labels1, prediction1)
union = np.logical_or(labels1, prediction1)

In [ ]:
kwarg_dict = {'CAFO Shed': {'cmap':'prism', 'alpha': 0.5},
              'Lagoon': {'cmap':'cool', 'alpha': 0.5},
              'BACKGROUND': {'cmap':'hot', 'alpha': 0.0}}

f = plt.figure()
a = f.add_subplot(1, 2, 1)
plt.imshow(inputs[2].permute(1, 2, 0).cpu().numpy())
plt.axis('off')
a = f.add_subplot(1, 2, 2)
plt.imshow(inputs[2].permute(1, 2, 0).cpu().numpy())
plt.imshow(labels[2][0].cpu().numpy(), **kwarg_dict['BACKGROUND'])
plt.imshow(labels[2][1].cpu().numpy(), **kwarg_dict['Lagoon'])
plt.imshow(labels[2][2].cpu().numpy(), **kwarg_dict['CAFO Shed'])    
plt.axis('off')
plt.show()

In [ ]:
inputs[0].shape

In [ ]:
example = inputs[1]
example1 = example.flip(-1)
example2 = example.flip(-2)
example3 = example.flip(-1).flip(-2)

In [ ]:
f = plt.figure(dpi=240)

fontdict = {'fontsize' : 14}

a1 = f.add_subplot(1, 2, 1)
plt.imshow(example.permute(1, 2, 0).cpu().numpy())    
a1.set_title("Original", fontdict)
plt.axis('off')

a2 = f.add_subplot(1, 2, 1)
plt.imshow(example1.permute(1, 2, 0).cpu().numpy())    
a2.set_title("Horizontal flip", fontdict)
plt.axis('off')

a3 = f.add_subplot(1, 2, 1)
plt.imshow(example2.permute(1, 2, 0).cpu().numpy())    
a3.set_title("Vertical flip", fontdict)
plt.axis('off')

a4 = f.add_subplot(1, 2, 1)
plt.imshow(example3.permute(1, 2, 0).cpu().numpy())    
a4.set_title("Both", fontdict)
plt.axis('off')

In [ ]:
print(UNet(3, 3)) # model architecture

In [ ]:
print(UNet(3, 2))

In [ ]:
print(sum(p.numel() for p in UNet(3, 3).parameters())) #Number of parameters
print(sum(p.numel() for p in UNet(3, 2).parameters())) #Number of parameters

In [ ]:
blah = torchvision.transforms.ColorJitter()
inputs2 = inputs + torch.normal(0, .3, size=(4, 3, 1024, 1024)).to(device)
inputs3 = blah(inputs)
#inputs2 = inputs + (torch.randn(inputs.size()) * .3).to(device)

In [ ]:
f, axarr = plt.subplots(1, 3)
axarr[0].imshow(inputs[2].permute(1, 2, 0).cpu().numpy())
axarr[1].imshow(inputs2[2].permute(1, 2, 0).cpu().numpy())
axarr[2].imshow(inputs3[2].permute(1, 2, 0).cpu().numpy())